This notebook translates to Italian the English terms in the datasets:
- Mayo
- MiniMayo
- ...

The translation is based on the MRCONSO file, provided by UMLS. Terms that cannot be translated using MRCONSO are translated using the most common translation among three automatic translation services or a random string in case of disagreement.


Authors: 
- F.A. Cardillo, francoalberto.cardillo@cnr.it
- F. Debole, (franca.debole@isti.cnr.it)
Date: 22 March 2024


__If you use this notebook or the resources it builds, please cite:__

__"Italian Word Embeddings for the Medical Domain", F.A. Cardillo, F. Debole. Proc. of the 2024 Joint Int. Conf. on Computational Linguistics, Language Resources and Evaluation (LREC-COLING 2024), Turin, Italy, May 20-25, 2024.__


The MIT License

Copyright 2024, Franco Alberto Cardillo, Franca Debole

Permission is hereby granted, free of charge, to any person obtaining a copy of this software and associated documentation files (the “Software”), to deal in the Software without restriction, including without limitation the rights to use, copy, modify, merge, publish, distribute, sublicense, and/or sell copies of the Software, and to permit persons to whom the Software is furnished to do so, subject to the following conditions:

The above copyright notice and this permission notice shall be included in all copies or substantial portions of the Software.

THE SOFTWARE IS PROVIDED “AS IS”, WITHOUT WARRANTY OF ANY KIND, EXPRESS OR IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY, FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM, OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN THE SOFTWARE.

<hr>

IN: evaluation resources (files with the term pair and similarity judgments)

OUT: `aux/cuis.csv`, dataframe with two columns "term", "cui". The column "cui" does not contain duplicates.
     notee: there might be multiple terms associated to the same CUI.

This cell reads all the evaluation resources and collects the pairs (CUI, term) from all the input files

In [1]:
import os
import pandas as pd
from posixpath import join

in_fld = "eval_resources/"
aux_fld = join("out", "aux_files") # output
os.makedirs(aux_fld, exist_ok=True)

files = sorted([fn for fn in os.listdir(in_fld) if fn.endswith(".csv")])

dfs = []
for fn in files:
    df = pd.read_csv(join(in_fld, fn))
    columns = ["Term1", "CUI1", "Term2", "CUI2"]
    if "mayo" in fn.lower():
        columns = [c.upper() for c in columns]
    # (Term, CUI)
    terms1 = df[columns[:2]]
    terms2 = df[columns[2:]]
    terms1.columns = ["term", "cui"]
    terms2.columns = terms1.columns
    terms = pd.concat([terms1, terms2], axis=0)
    terms["term"] = terms["term"].apply(lambda x: x.replace(".", " ").strip().lower())
    # eg one term "allergy." is spelled with and without a dot
    dfs.append(terms)

df = pd.concat(dfs, axis=0)
print("all terms:", len(df), "sample:")
display(df.sample(5))

# duplicated Concept Unique Identifiers (CUI)
udf = df.drop_duplicates(subset=["cui"])
print("unique cuis: ", len(udf))

# duplicated terms
dup_terms_iii = udf.duplicated(subset=["term"], keep=False)
print("duplicated terms with different cuis: ", len(udf[dup_terms_iii]))
display(udf[dup_terms_iii].sort_values(by=["term"]))

out_fn = join(aux_fld, "cuis.csv")
udf.to_csv(out_fn, index=False)
print("saved: ", out_fn)
print("done")

all terms: 4380 sample:


,term,cui
2,myocardium,C0027061
383,synthroid,C0040165
192,infertility,C0021359
208,headache,C0018681
23,angina,C0002962


unique cuis:  586
duplicated terms with different cuis:  18


,term,cui
77,allergy,C1998461
68,allergy,C0020517
5,calcification,C0175895
55,calcification,C0006660
3,diabetes,C0011849
25,diabetes,C0011847
20,laryngeal cancer,C0007107
81,laryngeal cancer,C0023055
100,osteoporosis,C0029456
54,osteoporosis,C1563292


saved:  out/aux_files/cuis.csv
done


IN: MRCONSO.RRF

OUT: `out/aux_files/MRCONSO_eng_ita.[csv|pkl]`, dataframe with the English and Italian subsets of MRCONSO.RRF, with column names (not specified in MRCONSO.RRF)

Read MRCONSO.RRF, add column names, restrict to Italian and English

In [6]:
from numpy import count_nonzero as nnz
import pandas as pd
from posixpath import join


aux_fld = join("out", "aux_files") # output
os.makedirs(aux_fld, exist_ok=True)


# reading MRCONSO takes approx. 40 secs
# >>> You need an UMLS licence to download the MRCONSO.RRF file
sep = "|"
df = pd.read_csv(join("external_resources", "MRCONSO.RRF"), sep=sep, header=None)
# drop last column, it should not be there
df = df.iloc[:, :-1]

# the rrf does not contain column names
columns = ["CUI", "LAT", "TS", "LUI", "STT", "SUI", "ISPREF", "AUI", "SAUI", "SCUI", "SDUI", "SAB", "TTY", "CODE", "STR", "SRL", "SUPPRESS", "CVF"]
df.columns = columns

print("MRCONSO.RRF")
display(df.head())
print("LANGUAGES:")
print(df.LAT.unique())

# keep only English and Italian
iii = (df.LAT == "ENG")  | (df.LAT == "ITA")
df = df.loc[iii, :]

# save both a csv and a pickle file
fn = join(aux_fld, "MRCONSO_eng_ita.csv")
df.to_csv(fn, index=False, sep=sep)
print("saved: ", fn)
fn = fn.replace(".csv", ".pkl")
df.to_pickle(fn)
print("saved: ", fn)

print("all done")

/tmp/ipykernel_167688/645654646.py:13: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(join("external_resources", "MRCONSO.RRF"), sep=sep, header=None)


MRCONSO.RRF


,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
0,C0000005,ENG,P,L0000005,PF,S0007492,Y,A26634265,NaN,M0019694,D012711,MSH,PEP,D012711,(131)I-Macroaggregated Albumin,0,N,256.0
1,C0000005,ENG,S,L0270109,PF,S0007491,Y,A26634266,NaN,M0019694,D012711,MSH,ET,D012711,(131)I-MAA,0,N,256.0
2,C0000005,FRE,P,L6220710,PF,S7133957,Y,A13433185,NaN,M0019694,D012711,MSHFRE,PEP,D012711,Macroagrégats d'albumine marquée à l'iode 131,3,N,NaN
3,C0000005,FRE,S,L6215648,PF,S7133916,Y,A27488794,NaN,M0019694,D012711,MSHFRE,ET,D012711,MAA-I 131,3,N,NaN
4,C0000005,FRE,S,L6215656,PF,S7133956,Y,A27614225,NaN,M0019694,D012711,MSHFRE,ET,D012711,Macroagrégats d'albumine humaine marquée à l'i...,3,N,NaN


LANGUAGES:
['ENG' 'FRE' 'SWE' 'CZE' 'FIN' 'GER' 'ITA' 'JPN' 'POL' 'POR' 'RUS' 'SPA'
 'SCR' 'NOR' 'DUT' 'LAV' 'ARA' 'GRE' 'HUN' 'KOR' 'BAQ' 'DAN' 'HEB' 'CHI'
 'EST' 'TUR' 'UKR']
saved:  out/aux_files/MRCONSO_eng_ita.csv
saved:  out/aux_files/MRCONSO_eng_ita.pkl
all done


IN: `out/aux_files/MRCONSO_eng_ita.pkl`, `out/aux_files/cuis.csv`

OUT: `out/cuis_ita.csv`, first subset of CUIs translated to Italian.

Try to associate an Italian term to each CUI in the input file `...cuis.csv` using `...MRCONSO_eng_ita`

In [8]:
# Now translate cuis.csv (CUI,ENG) to (CUI,ENG,ITA,SOURCE)
from numpy import count_nonzero as nnz
import pandas as pd
from posixpath import join

aux_fld = join("out", "aux_files")
# approx. 17 secs
in_fn = join(aux_fld, "MRCONSO_eng_ita.pkl")
df = pd.read_pickle(in_fn)

cui_df = pd.read_csv(join(aux_fld, "cuis.csv"))
cui_df["cui"] = cui_df.cui.apply(lambda x: str(x))
print("CUI.CSV:", cui_df.shape, len(cui_df))
display(cui_df)

# display counts by language
print("COUNTS BY LANGUAGE")
display(df.groupby("LAT").count())

# dataframe with Italian only
ita = df[df.LAT == "ITA"].copy()
ita["CUI"] = ita.CUI.apply(lambda x: str(x))

ita["ISPREF_sort"] = ita["ISPREF"].apply(lambda x: 0 if x == 'Y' else 1)
# ita["TTY_sort"] = ita["TTY"].apply(lambda x: 0 if x == 'PT' else 1)

# vocabularies: ['MSHITA', 'MDRITA', 'LNC-IT-IT', 'ICPCITA', 'MTHMSTITA']
def vocab_sort(x):
    if x == "MSHITA":
        return 0
    elif x == "MDRITA":
        return 1
    elif x == "MTHMSTITA":
        return 2
    else:
        return 3


ita["SAB_sort"] = ita["SAB"].apply(lambda x: vocab_sort(x))

def tty_order(tty):
    if tty == 'PT':
        return 0
    elif tty == 'MH':
        return 1
    elif tty == "ET":
        return 2
    else:
        return 3
    

ita['TTY_sort'] = ita['TTY'].apply(lambda x: tty_order(x))

print("ITA")
display(ita.head())

print("Italian SAB - vocabularies")
print(ita.SAB.unique())

# counts by vocabulary and ISPREF field
print("ITA - COUNTS BY SAB")
display(ita.groupby("SAB").count().sort_values(by=["CUI"], ascending=False))
print("ITA - COUNTS BY ISPREF")
display(ita.groupby("ISPREF").count().sort_values(by=["CUI"], ascending=False))

# join cui_df and ita on cui
merged = cui_df.merge(ita, left_on="cui", right_on="CUI", how="left")
cuis_without_translation = merged.STR.isna()

# look for translations in the following vocabularies in ORDER (priority)
# ord_sources = ['MSHITA', 'MDRITA', 'LNC-IT-IT', 'ICPCITA', 'MTHMSTITA']
# ord_sources = { s:i for i, s in enumerate(ord_sources) }

def sort_group(g):
    # return g.sort_values(by=["SAB", "ISPREF"], key=lambda x: x.map(ord_sources), ascending=True).sort_values(by=["ISPREF"], ascending=False).head(1)
    return g.sort_values(by=["SAB_sort", "TTY_sort", "ISPREF_sort"], ascending=True).head(1)


out = merged.groupby(["cui"], group_keys=False, sort=False).apply(sort_group)
display(out.groupby("ISPREF").count())
out = out[ ["cui", "term", "STR", "SAB" ] ].copy()
out.columns = ["cui", "term", "term_it", "sab"]
print("OUT,", out.shape)
display(out)

n_na = nnz(out.term_it.isna())
print(f"{n_na} terms still without translation")

# check, see above
assert n_na == nnz(cuis_without_translation), "check n_na"
assert len(out) == len(cui_df), "check dims"

print("TRANSLATIONS GROUPED BY SOURCE DICTIONARY")
display(out.groupby("sab").count())
out.term_it = out.term_it.apply(lambda x: x.lower().strip() if not pd.isna(x) else x)

print("OUTPUT DATAFRAME THAT WILL BE SAVED")
display(out)

fn = join(aux_fld, "cuis_ita.csv")
out.to_csv(fn, index=False)
print("saved: ", fn)
print("all done")



CUI.CSV: (586, 2) 586


,term,cui
0,difficulty walking,C0311394
1,rheumatoid nodule,C0035450
2,hand splint,C0409162
3,diabetes,C0011849
4,portal hypertension,C0020541
...,...,...
581,vaccinia,C0042214
582,tums,C0006681
583,corkscrewing,C0231618
584,zetia,C1142985


COUNTS BY LANGUAGE


,CUI,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF
LAT,,,,,,,,,,,,,,,,,
ENG,8510801,8510801,8510801,8510801,8510801,8510801,8510801,1391613,6307593,1778008,8510801,8500557,8510790,8510773,8510801,8510801,5372002
ITA,249494,249494,249494,249494,249494,249494,249494,0,142685,163818,249494,249494,249494,249494,249494,249494,0


ITA


...

64 terms still without translation
TRANSLATIONS GROUPED BY SOURCE DICTIONARY


,cui,term,term_it
sab,,,
MDRITA,64,64,64
MSHITA,458,458,458


OUTPUT DATAFRAME THAT WILL BE SAVED


,cui,term,term_it,sab
2,C0311394,difficulty walking,difficoltà nella deambulazione,MSHITA
10,C0035450,rheumatoid nodule,nodulo reumatoide,MSHITA
11,C0409162,hand splint,NaN,NaN
15,C0011849,diabetes,diabete mellito,MSHITA
20,C0020541,portal hypertension,ipertensione portale,MSHITA
...,...,...,...,...
2444,C0042214,vaccinia,pustola vaccinica,MSHITA
2446,C0006681,tums,carbonato di calcio,MSHITA
2447,C0231618,corkscrewing,NaN,NaN
2448,C1142985,zetia,ezetimibe,MSHITA


saved:  out/aux_files/cuis_ita.csv
all done


IN: `out/aux_files/cuis_ita.csv`

OUT: `out/aux_files/cuis_ita2.csv`, as the IN file with more translated CUIs.

Some CUIs are not present in the Italian subset of MRCONSO and are not translated in `...cuis_ita.csv`. However, `cuis.csv` contains some duplicated English terms associated to different CUIs, some of which are in `MRCONSO_eng_ita` and have been translated. For the CUIs with duplicated terms, we here associate the available translation to the untranslated CUI.

If there are multiple English terms t2 == t1 (same term, multiple CUIS), select a random one.
With our files, for each English t1 there is one duplicate at most.

In [10]:
# find duplicated English terms in cuis_ita.csv
import pandas as pd
from numpy import count_nonzero as nnz

aux_fld = join("out", "aux_files")
ita = pd.read_csv(join(aux_fld, "cuis_ita.csv"))

dup_terms_iii = ita.duplicated(subset=["term"], keep=False)
print("DUPLICATED TERMS in cuis_ita.csv: same term with different CUIs", len(ita[dup_terms_iii]))
display(ita[dup_terms_iii].sort_values(by=["term"]))

def fill_na_translation(g):
    na = g.term_it.isna()
    if (nnz(na) > 0) and len(g)>1:
        g.loc[na, "term_it"] = g.loc[~na, "term_it"].sample(1).iloc[0]
        g.loc[na, "sab"] = "dup"
    return g 
out = ita.groupby(["term"], group_keys=False, sort=False).apply(fill_na_translation)

dup_terms_iii = out.duplicated(subset=["term"], keep=False)
print("DUPLICATED TERMS: NO NaNs should be present in what follows")
display(out[dup_terms_iii].sort_values(by=["term"]))

assert nnz(out[dup_terms_iii].term_it.isna()) == 0, "check NaNs"

fn = join(aux_fld, "cuis_ita2.csv")
out.to_csv(fn, index=False)
print("saved: ", fn)
print("Dimensions of cuis_ita2 ", out.shape)
print("all done")

DUPLICATED TERMS in cuis_ita.csv: same term with different CUIs 18


,cui,term,term_it,sab
266,C1998461,allergy,NaN,NaN
155,C0020517,allergy,ipersensibilità,MSHITA
185,C0175895,calcification,NaN,NaN
144,C0006660,calcification,calcificazione fisiologica,MSHITA
3,C0011849,diabetes,diabete mellito,MSHITA
229,C0011847,diabetes,NaN,NaN
202,C0007107,laryngeal cancer,tumore laringeo,MSHITA
165,C0023055,laryngeal cancer,neoplasie della laringe,MSHITA
181,C0029456,osteoporosis,osteoporosi,MSHITA
143,C1563292,osteoporosis,NaN,NaN


DUPLICATED TERMS: NO NaNs should be present in what follows


,cui,term,term_it,sab
266,C1998461,allergy,ipersensibilità,dup
155,C0020517,allergy,ipersensibilità,MSHITA
185,C0175895,calcification,calcificazione fisiologica,dup
144,C0006660,calcification,calcificazione fisiologica,MSHITA
3,C0011849,diabetes,diabete mellito,MSHITA
229,C0011847,diabetes,diabete mellito,dup
202,C0007107,laryngeal cancer,tumore laringeo,MSHITA
165,C0023055,laryngeal cancer,neoplasie della laringe,MSHITA
181,C0029456,osteoporosis,osteoporosi,MSHITA
143,C1563292,osteoporosis,osteoporosi,dup


saved:  out/aux_files/cuis_ita2.csv
Dimensions of cuis_ita2  (586, 4)
all done


IN: `MRCONSO_eng_ita`, `cuis_ita2.csv`

OUT: `out/aux_fld/cuis_ita3.csv`, that corresponds to `cui_ita2.csv` with the additional column `alt_en`; `out/aux_files/remaining_terms.txt`, list of terms to translate

At this point some English terms are still without a translation because their corresponding CUI are not in the Italian subset of MRCONSO.

Each untranslated English term will be mapped to its most common variation using the fields LUI, SUI and ISPREF as defined in MRCONSO. 

This step is needed to translate English terms that are way too specific to the American context:
- for example, "banana bag" for "multivitamin"
- some drug names are mapped to their main active ingredient
The substitution affects only a few terms and leaves unchanged the majority of them.

We will translate the terms in the column `alt_en`using standard web-based translation services (google translate, deepl, gpt3.5).

In [12]:
# for each term that has not yet a translation
#   - get the CUI
#   - get the LUIs associated to the LUI
#   - choose the LUI with the highest size
#   - get the STR from the LUI with ISPREF set to true

import numpy
from numpy import count_nonzero as nnz
import pandas as pd

aux_fld = join("out", "aux_files")
cuis = pd.read_csv(join(aux_fld,"cuis_ita2.csv"))
print("CUIS_ITA2, head")
display(cuis.head(5))

eng = pd.read_pickle(join(aux_fld, "MRCONSO_eng_ita.pkl"))
eng = eng[eng.LAT == "ENG"]

eng["LUI_SIZE"] = eng.groupby(["CUI", "LUI"])["LUI"].transform("count")
eng["SUI_SIZE"] = eng.groupby(["CUI", "LUI", "SUI"])["SUI"].transform("count")

na = cuis.term_it.isna()
print("terms without translation:", nnz(na))

def fill_na_luis(row):
    cui = row.cui
    term = eng[eng.CUI == cui].sort_values(by=["LUI_SIZE", "SUI_SIZE", "ISPREF"], ascending=False).head(1)["STR"]
    if(len(term) == 0):
        print("***", cui, "not found")
        term = "cui_not_found"
    else:
        term = term.iloc[0]
    return term


cuis.loc[na, "alt_en"] = cuis[na].apply(lambda x: fill_na_luis(x) if pd.isna(x.term_it) else np.nan, axis=1)
display(cuis[na])

n_cui_not_found = nnz(cuis.alt_en == "cui_not_found")
print("cui_not_found:", n_cui_not_found)

cuis.loc[cuis.alt_en == "cui_not_found", "alt_en"] = cuis.loc[cuis.alt_en == "cui_not_found", "term"]
cuis.loc[na, "alt_en"] = cuis.loc[na, "alt_en"].apply(lambda x: x.lower().replace("*", " ").strip())
display(cuis.loc[na])

fn = join(aux_fld, "cuis_ita3.csv")
cuis.to_csv(fn, index=False)
print("saved: ", fn)

fn = join(aux_fld,"remaining_terms.txt")  # terms to translate
with open(fn, "w") as fout:
    fout.write("\n".join(cuis.loc[na, "alt_en"].tolist()))

print("saved: ", fn)
print("all done")

CUIS_ITA2, head


,cui,term,term_it,sab
0,C0311394,difficulty walking,difficoltà nella deambulazione,MSHITA
1,C0035450,rheumatoid nodule,nodulo reumatoide,MSHITA
2,C0409162,hand splint,NaN,NaN
3,C0011849,diabetes,diabete mellito,MSHITA
4,C0020541,portal hypertension,ipertensione portale,MSHITA


terms without translation: 58
*** C2267026 not found
*** C1527356 not found
*** C0011127 not found
*** C0009951 not found
*** C0031763 not found
*** C0947651 not found
*** C0557875 not found


,cui,term,term_it,sab,alt_en
2,C0409162,hand splint,NaN,NaN,Splinting of hand
14,C2267026,hmg co a reductase inhibitor,NaN,NaN,cui_not_found
19,C0231736,drawer sign,NaN,NaN,Drawer sign
20,C0332536,laxity,NaN,NaN,Laxity
22,C0457086,morning stiffness,NaN,NaN,Morning stiffness
27,C0429103,t wave,NaN,NaN,T wave
28,C1510420,cavitation,NaN,NaN,Cavitation
56,C0333997,lymphoid hyperplasia,NaN,NaN,Lymphoid hyperplasia
64,C0013394,dysparunia,NaN,NaN,Dyspareunia
104,C1444657,contraindicated,NaN,NaN,Contraindicated


cui_not_found: 7


,cui,term,term_it,sab,alt_en
2,C0409162,hand splint,NaN,NaN,splinting of hand
14,C2267026,hmg co a reductase inhibitor,NaN,NaN,hmg co a reductase inhibitor
19,C0231736,drawer sign,NaN,NaN,drawer sign
20,C0332536,laxity,NaN,NaN,laxity
22,C0457086,morning stiffness,NaN,NaN,morning stiffness
27,C0429103,t wave,NaN,NaN,t wave
28,C1510420,cavitation,NaN,NaN,cavitation
56,C0333997,lymphoid hyperplasia,NaN,NaN,lymphoid hyperplasia
64,C0013394,dysparunia,NaN,NaN,dyspareunia
104,C1444657,contraindicated,NaN,NaN,contraindicated


saved:  out/aux_files/cuis_ita3.csv
saved:  out/aux_files/remaining_terms.txt
all done


We use three web-based translation services:
- google translation: the translation is performed programmatically (no API key required)
  OUT: `out/aux_files/remaining_terms_google-it.txt`
- deepl: the translation is performed programmatically (an API key is required)
  OUT: `out/aux_files/remaining_terms_deepl-it.txt`
- GPT3.5: the translation needs to be done manually
  OUT: `out/aux_files/remaining_terms_gpt35-it.txt`

In [15]:
# google
from googletrans import Translator
from posixpath import join

aux_fld = join("out", "aux_files")

translator = Translator()
with open(join(aux_fld, "remaining_terms.txt"), "r") as fin:
    unique_terms = [l.strip() for l in fin.readlines()]

it_terms = [ translator.translate(t, src="en", dest="it").text for t in unique_terms]
fn = join(aux_fld, "remaining_terms_google-it.txt")
with open(fn, "w") as fout:
    fout.write("\n".join(it_terms))

print('google translations done, saved:', fn)
print("all done")

google translations done, saved: out/aux_files/remaining_terms_google-it.txt
all done


In [16]:
# deepl
import deepl
from posixpath import join
aux_fld = join("out", "aux_files")

with open(join(aux_fld, "remaining_terms.txt"), "r") as fin:
    unique_terms = [l.strip() for l in fin.readlines()]

# TODO: add your deepl auth key below
auth_key = "YOUR DEEPL KEY HERE"
translator = deepl.Translator(auth_key)

# this is fast
it_terms = translator.translate_text("\n".join(unique_terms), target_lang="IT")
fn = join(aux_fld, "remaining_terms_deepl-it.txt")
with open(fn, "w") as fout:
    fout.write(it_terms.text)

print("deepl translations done, saved:", fn)
print("all done")

deepl translations done, saved: out/aux_files/remaining_terms_deepl-it.txt
all done


### GPT
there is a third file with ChatGPT 3.5. translation has been done manually with the prompt "..." on 100 items per time.

IN: `out/aux_files/remaining_terms_*-it.txt`

OUT: `out/aux_files/remaining_terms_translations.csv`

Merge the three translations into a single dataframe 

- choosing the Italian translation when at least 2 out of the 3 services returned the same string 
- leaving the other terms (3 different translations) untranslated

In [20]:
import os
import pandas as pd
from posixpath import join

aux_fld = join("out", "aux_files")

# read all_terms
with open(join(aux_fld, "remaining_terms.txt"), "r") as fin:
    unique_terms = [l.strip() for l in fin.readlines()]

# find translations, i.e. files ending with "-it.txt"
files = [fn for fn in os.listdir(aux_fld) if fn.endswith("-it.txt")]
terms = []
for fn in files:
    with open(join(aux_fld, fn), "r") as fin:
        terms.append([l.strip().lower() for l in fin.readlines()])
# filename is of the form "all_terms_{providers}-it.txt"
# extract the provider name
providers = [fn.split("_")[2].split("-")[0] for fn in files]
trans = {}
trans["original"] = unique_terms

for l, provider in zip(terms, providers):
    trans[provider] = l

for k, l in trans.items():
    print(f"{k}: {len(l)}")

# create dataframe
df = pd.DataFrame.from_dict(trans)
df["n"] = df.apply(lambda x: len(set(x[1:].values)), axis=1)
df["agree"] = (df.n == 1)
# select the most common value where n is 2
df["translation"] = df.apply(lambda x: x.value_counts().index[0] if x.n != 3 else None, axis=1)

display(df.value_counts("n"))
print("terms without a majority translation, count", len(df.loc[df.n == 3]))
display(df.loc[df.n == 3])

fn = join(aux_fld, "remaining_terms_translations.csv")
df.to_csv(fn, index=False)
print("saved: ", fn)
print("all done")

original: 58
gpt35: 58
deepl: 58
google: 58


n
1    40
2    11
3     7
Name: count, dtype: int64

terms without a majority translation, count  7


,original,gpt35,deepl,google,n,agree,translation
1,hmg co a reductase inhibitor,inibitore dell'hmg coa reduttasi,hmg co a inibitore della reduttasi,inibitore della hmg coa reduttasi,3,False,None
11,entire cranial nerve,nervo cranico,nervo cranico intero,intero nervo cranico,3,False,None
23,entire knee meniscus,menisco del ginocchio,menisco del ginocchio intero,intero menisco del ginocchio,3,False,None
34,maculopapule,maculopapula,maculopapule,maculopapulare,3,False,None
36,cromolyn,cromoglicato,cromolinio,cromolina,3,False,None
47,camelpox,variola del cammello,camelpox,vaiolo del cammello,3,False,None
54,catch,accenno,cattura,presa,3,False,None


saved:  out/aux_files/remaining_terms_translations.csv
all done


IN: `remaining_terms_translations.csv`, `cuis_ita3.csv` in `out/aux_files`

OUT: `out/aux_files/cuis_ita4.csv`, file with all CUIs associated to an Italian term

For the terms still without translatins (i.e. we received three different translations from the three translation services), choose a translation randomly.

Then, prepare the final file with all the translations.

In [30]:
# now use web translation to complete the set of translated termss

import pandas as pd
import os
from posixpath import join
from numpy import count_nonzero as nnz
import numpy as np
import random

aux_fld = join("out", "aux_files")

fn = join(aux_fld, "remaining_terms_translations.csv")
translations = pd.read_csv(fn)
print("TRANSLATIONS")
display(translations.head())
# please note: translations with "n" <=2 have a translated term, otherwise the have NaN

cols = ["gpt35", "google", "deepl"]
def fill_na_trans(row):
    values = row[cols]
    h = hash("".join(values)) % (2**32)
    random.seed(h)
    i = random.randint(0, 2)
    sample = values.iloc[i]
    row["translation"] = sample
    return row

# choose a random translation for terms with n=3 different translations
translations = translations.apply(lambda x: fill_na_trans(x) if x.n == 3 else x, axis=1)
na = translations.translation.isna()
# all terms should be translated at this point
assert nnz(na) == 0, "check fill_na"

fn = join(aux_fld,"cuis_ita3.csv")
cuis = pd.read_csv(fn)

# use term in alt_en if available
cuis["term"] = cuis.apply(lambda x: x.alt_en if not pd.isna(x.alt_en) else x.term, axis=1)
# cuis = cuis.drop_columns(["alt_en"])
# display(cuis.head(10))

iii = cuis.term_it.isna()
print("CUIS WITHOUT TRANSLATION:", nnz(iii))
display(cuis[cuis.term_it.isna()].sort_values(by=["term"]))

translations = translations[["original", "translation"]]
merged = cuis.merge(translations, left_on="term", right_on="original", how="left")
print("(temp result) SHOWING UNTRANSLATED TERMS:")
display(merged[cuis.term_it.isna()].sort_values(by=["term"]))

iii = cuis.term_it.isna()
# fill value in column term_it at indeces iii
merged.loc[iii, "term_it"] = merged.loc[iii, "translation"]
merged.loc[iii, "sab"] = "web"
merged = merged.drop(columns=["original", "translation", "alt_en"])

# some checks
assert cuis.shape[0] == merged.shape[0], "result with a wrong number of rows"
assert cuis.shape[1] == merged.shape[1]+1, "result with a wrong number of columns"
iii = merged.term_it.isna()
assert nnz(iii) == 0, "fill empty translation failed"

print("final result, all terms translated")
display( merged.groupby("sab").sample(3))
display( merged[["cui", "sab"]].groupby("sab").count())

fn = join(aux_fld, "cuis_ita4.csv")
merged.to_csv(fn, index=False)
print("saved: ", fn)
print("all done")

TRANSLATIONS


,original,gpt35,deepl,google,n,agree,translation
0,splinting of hand,steccatura della mano,steccatura della mano,steccatura della mano,1,True,steccatura della mano
1,hmg co a reductase inhibitor,inibitore dell'hmg coa reduttasi,hmg co a inibitore della reduttasi,inibitore della hmg coa reduttasi,3,False,NaN
2,drawer sign,segno del cassetto,segno del cassetto,segno del cassetto,1,True,segno del cassetto
3,laxity,lassità,lassità,lassismo,2,False,lassità
4,morning stiffness,rigidità mattutina,rigidità mattutina,rigidità mattutina,1,True,rigidità mattutina


CUIS WITHOUT TRANSLATION: 58


,cui,term,term_it,sab,alt_en
194,C0009814,acquired stenosis,NaN,NaN,acquired stenosis
322,C0995182,aloe vera,NaN,NaN,aloe vera
110,C0221434,bony sclerosis,NaN,NaN,bony sclerosis
461,C0357929,calamine,NaN,NaN,calamine
429,C0276185,camelpox,NaN,NaN,camelpox
396,C0717550,candesartan,NaN,NaN,candesartan
246,C0858529,cardialgia,NaN,NaN,cardialgia
540,C0231617,catch,NaN,NaN,catch
28,C1510420,cavitation,NaN,NaN,cavitation
104,C1444657,contraindicated,NaN,NaN,contraindicated


(temp result) SHOWING UNTRANSLATED TERMS:


,cui,term,term_it,sab,alt_en,original,translation
194,C0009814,acquired stenosis,NaN,NaN,acquired stenosis,acquired stenosis,stenosi acquisita
322,C0995182,aloe vera,NaN,NaN,aloe vera,aloe vera,aloe vera
110,C0221434,bony sclerosis,NaN,NaN,bony sclerosis,bony sclerosis,sclerosi ossea
461,C0357929,calamine,NaN,NaN,calamine,calamine,calamina
429,C0276185,camelpox,NaN,NaN,camelpox,camelpox,vaiolo del cammello
396,C0717550,candesartan,NaN,NaN,candesartan,candesartan,candesartan
246,C0858529,cardialgia,NaN,NaN,cardialgia,cardialgia,cardialgia
540,C0231617,catch,NaN,NaN,catch,catch,cattura
28,C1510420,cavitation,NaN,NaN,cavitation,cavitation,cavitazione
104,C1444657,contraindicated,NaN,NaN,contraindicated,contraindicated,controindicato


final result, all terms translated


,cui,term,term_it,sab
508,C0332573,macule,macula,MDRITA
272,C0152447,urethrorrhea,secrezione uretrale,MDRITA
444,C0178664,glomerulosclerosis,glomerulosclerosi,MDRITA
477,C0003564,aphonia,afonia,MSHITA
162,C0007102,colon cancer,cancro del colon,MSHITA
134,C0149745,mouth ulcer,ulcera orale,MSHITA
185,C0175895,calcification,calcificazione fisiologica,dup
417,C1883552,weakness,astenia,dup
229,C0011847,diabetes,diabete mellito,dup
370,C0231851,fish-mouthing,bocca di pesce,web


,cui
sab,
MDRITA,64
MSHITA,458
dup,6
web,58


saved:  out/aux_files/cuis_ita4.csv
all done


## FINAL STEP

IN: `out/aux_files/cuis_ita4.csv`, `eval_resources/*`

OUT: translated resources (Mayo, UMNSRS files) in `out/`

In [32]:
import pandas as pd
import os
from posixpath import join
from numpy import count_nonzero as nnz

fld = "eval_resources/"
aux_fld = join("out", "aux_files")
out_fld = join("out", fld)
os.makedirs(out_fld, exist_ok=True)

csvs = [fn for fn in os.listdir(fld) if fn.endswith(".csv")]
print("INPUT FILES")
for fn in csvs:
    print(fn)

fn = join(aux_fld, "cuis_ita4.csv")
cuis = pd.read_csv(fn)

cui2ita = {}
for cui, term_it in zip(cuis.cui, cuis.term_it):
    cui2ita[cui] = term_it

# df = df.merge(cuis, left_on="cui1", right_on="cui", how="left").drop(columns=drop_from_cuis).rename(columns={"ita": "term1_it"})


def process_df(df):
    term1 = df.cui1.map(cui2ita)
    term2 = df.cui2.map(cui2ita)
    df["term1_it"] = term1
    df["term2_it"] = term2
    return df

for fn in csvs:
    df2 = pd.read_csv(join(fld, fn))
    df = df2.copy()
    df.columns = [c.lower() for c in df.columns]
    df = process_df(df)
    assert len(df) == len(df2), "check rows in df, df2"
    df2["term1_it"] = df.term1_it
    df2["term2_it"] = df.term2_it
    
    na = df2.term1_it.isna() | df2.term2_it.isna()
    assert nnz(na) == 0, "check cui to term, there are nans"
    fn2 = fn.replace(".csv", "_it.csv")
    out_fn = join(out_fld, fn2)
    print("saving in:", out_fn)
    df2.to_csv(out_fn, index=False)
    print("saved: ", fn2)
    
print("all done")

INPUT FILES
MayoSRS.csv
UMNSRS_relatedness.csv
MiniMayoSRS.csv
UMNSRS_relatedness_mod458_word2vec.csv
UMNSRS_similarity_mod449_word2vec.csv
UMNSRS_similarity.csv
saving in: out/eval_resources/MayoSRS_it.csv
saved:  MayoSRS_it.csv
saving in: out/eval_resources/UMNSRS_relatedness_it.csv
saved:  UMNSRS_relatedness_it.csv
saving in: out/eval_resources/MiniMayoSRS_it.csv
saved:  MiniMayoSRS_it.csv
saving in: out/eval_resources/UMNSRS_relatedness_mod458_word2vec_it.csv
saved:  UMNSRS_relatedness_mod458_word2vec_it.csv
saving in: out/eval_resources/UMNSRS_similarity_mod449_word2vec_it.csv
saved:  UMNSRS_similarity_mod449_word2vec_it.csv
saving in: out/eval_resources/UMNSRS_similarity_it.csv
saved:  UMNSRS_similarity_it.csv
all done
